In [1]:
from pathlib import Path
from random import random

import polars as pl

from extractors import Extractor

# Extract tar file

In [2]:
DATA_DIR = 'data'
FILE_NAME = 'imputed_goiener_v7.tar.zst'
EXTRACTED_DIR = Path(DATA_DIR, FILE_NAME.split('.')[0]) # imputed_goiener_v7

METADATA = Path(DATA_DIR, 'metadata.csv')

In [3]:
if not EXTRACTED_DIR.exists():
    extractor = Extractor(Path(DATA_DIR, FILE_NAME), DATA_DIR)

    extractor.decompress_tzst()

# Metadata standardization

In [4]:
metadata = pl.read_csv(METADATA, try_parse_dates=True, schema_overrides={'cnae':pl.String, 'codigo_postal':pl.String, 'fecha_alta':pl.String, 'fecha_baja':pl.String, 'p1_kw':pl.String, 'p2_kw':pl.String, 'p3_kw':pl.String, 'p4_kw':pl.String, 'p5_kw':pl.String, 'p6_kw':pl.String})
metadata

cups,fecha_alta,fecha_baja,p1_kw,p2_kw,p3_kw,codigo_postal,cnae,tarifa_atr,p4_kw,p5_kw,p6_kw
str,str,str,str,str,str,str,str,str,str,str,str
"""bd2b4dc5f0736640d40f6a88ce0db7…","""2017-06-24""","""NA""","""4.4""","""NA""","""NA""","""48993""","""9820""","""NA""","""NA""","""NA""","""NA"""
"""68acb4e01f277604d8729ecd4e4449…","""2017-11-01""","""NA""","""3.45""","""NA""","""NA""","""48300""","""9820""","""NA""","""NA""","""NA""","""NA"""
"""3d9675cee4194df1bed1afee12c257…","""2017-12-29""","""NA""","""3.45""","""NA""","""NA""","""31191""","""9820""","""NA""","""NA""","""NA""","""NA"""
"""e699f5789248530ba2ceab04b9da04…","""2017-09-13""","""NA""","""5.75""","""NA""","""5.75""","""20018""","""9820""","""NA""","""NA""","""NA""","""NA"""
"""80d96612acc08c3766d51bd94aeccf…","""2017-06-29""","""NA""","""4.4""","""NA""","""NA""","""48007""","""9820""","""NA""","""NA""","""NA""","""NA"""
…,…,…,…,…,…,…,…,…,…,…,…
"""899dda294b0dfab08e786eaccd63f2…","""2021-06-01""","""NA""","""3.45""","""3.45""","""NA""","""21200""","""9820""","""2.0TD""","""NA""","""NA""","""NA"""
"""f275f372d4bb766f56a6529c77dcb1…","""2021-06-01""","""2022-05-02""","""4.6""","""4.6""","""NA""","""42150""","""9820""","""2.0TD""","""NA""","""NA""","""NA"""
"""428398a84ce0522ea028cdeb93cf52…","""2021-07-02""","""2022-05-04""","""4.6""","""4.6""","""NA""","""42002""","""9821""","""2.0TD""","""NA""","""NA""","""NA"""


In [5]:
metadata_standardized = metadata.with_columns(
    pl.col("cups").alias("id"),
    pl.when(pl.col("fecha_alta").str.contains("/"))
      .then(
          pl.col("fecha_alta")
            .str.strptime(pl.Date, "%d/%m/%Y", strict=False)
            .dt.strftime("%Y-%m-%d")
      )
      .otherwise(
          pl.when(pl.col("fecha_alta").str.contains("-"))
            .then(pl.col("fecha_alta"))
            .otherwise(None)
      ).alias("start_date")
    .cast(pl.Date),
    pl.when(pl.col("fecha_baja").str.contains("/"))
      .then(
          pl.col("fecha_baja")
            .str.strptime(pl.Date, "%d/%m/%Y", strict=False)
            .dt.strftime("%Y-%m-%d")
      )
      .otherwise(
          pl.when(pl.col("fecha_baja").str.contains("-"))
            .then(pl.col("fecha_baja"))
            .otherwise(None)
      ).alias("end_date")
    .cast(pl.Date),
    pl.when(pl.col("p1_kw") == "NA")
      .then(pl.lit(None).cast(pl.Float64))
      .otherwise(pl.col("p1_kw").cast(pl.Float64, strict=False)).alias("p1_kw"),
    # pl.when(pl.col("p2_kw") == "NA")
    #   .then(pl.lit(None).cast(pl.Float64))
    #   .otherwise(pl.col("p2_kw").cast(pl.Float64, strict=False)).alias("p2_kw"),
    # pl.when(pl.col("p3_kw") == "NA")
    #   .then(pl.lit(None).cast(pl.Float64))
    #   .otherwise(pl.col("p3_kw").cast(pl.Float64, strict=False)).alias("p3_kw"),
    # pl.when(pl.col("p4_kw") == "NA")
    #   .then(pl.lit(None).cast(pl.Float64))
    #   .otherwise(pl.col("p4_kw").cast(pl.Float64, strict=False)).alias("p4_kw"),
    # pl.when(pl.col("p5_kw") == "NA")
    #   .then(pl.lit(None).cast(pl.Float64))
    #   .otherwise(pl.col("p5_kw").cast(pl.Float64, strict=False)).alias("p5_kw"),
    # pl.when(pl.col("p6_kw") == "NA")
    #   .then(pl.lit(None).cast(pl.Float64))
    #   .otherwise(pl.col("p6_kw").cast(pl.Float64, strict=False)).alias("p6_kw"),

    pl.when(pl.col("codigo_postal") == "NA")
      .then(None)
      .otherwise(pl.col("codigo_postal").cast(pl.String, strict=False)).alias("postal_code"),
    pl.when(pl.col("cnae") == "NA")
      .then(None)
      .otherwise(pl.col("cnae").cast(pl.String, strict=False)).alias("cnae"),
    pl.when(pl.col("tarifa_atr") == "NA")
      .then(None)
      .otherwise(pl.col("tarifa_atr").cast(pl.String, strict=False)).alias("tarriff"),
).drop(["cups", "fecha_alta", "fecha_baja", "codigo_postal", "tarifa_atr"])

metadata_standardized = metadata_standardized.select([
    pl.col("id"),
    pl.col("start_date"),
    pl.col("end_date"),
    pl.col("cnae"),
    pl.col("postal_code"),
    pl.col("p1_kw"),
    # pl.col("p2_kw"),
    # pl.col("p3_kw"),
    # pl.col("p4_kw"),
    # pl.col("p5_kw"),
    # pl.col("p6_kw"),
    pl.col("tarriff"),
])

metadata_standardized = metadata_standardized.unique(maintain_order=True)

metadata_standardized

id,start_date,end_date,cnae,postal_code,p1_kw,tarriff
str,date,date,str,str,f64,str
"""bd2b4dc5f0736640d40f6a88ce0db7…",2017-06-24,null,"""9820""","""48993""",4.4,null
"""68acb4e01f277604d8729ecd4e4449…",2017-11-01,null,"""9820""","""48300""",3.45,null
"""3d9675cee4194df1bed1afee12c257…",2017-12-29,null,"""9820""","""31191""",3.45,null
"""e699f5789248530ba2ceab04b9da04…",2017-09-13,null,"""9820""","""20018""",5.75,null
"""80d96612acc08c3766d51bd94aeccf…",2017-06-29,null,"""9820""","""48007""",4.4,null
…,…,…,…,…,…,…
"""36e3f2fbf4a1643e372809557db219…",2021-06-01,2022-04-25,"""6810""","""21200""",4.6,"""2.0TD"""
"""f275f372d4bb766f56a6529c77dcb1…",2021-06-01,2022-05-02,"""9820""","""42150""",4.6,"""2.0TD"""
"""428398a84ce0522ea028cdeb93cf52…",2021-07-02,2022-05-04,"""9821""","""42002""",4.6,"""2.0TD"""


In [6]:
# metadata_standardized.write_csv('data/metadata_standardized.csv')

In [7]:
# metadata_standardized = pl.read_csv('data/metadata_standardized.csv', try_parse_dates=True, schema_overrides={'cnae':pl.String, 'postal_code':pl.String, 'start_date':pl.Date, 'end_date':pl.Date, 'p1_kw':pl.Float64, 'p2_kw':pl.Float64, 'p3_kw':pl.Float64, 'p4_kw':pl.Float64, 'p5_kw':pl.Float64, 'p6_kw':pl.Float64})
# metadata_standardized

# Analyze metadata

In [8]:
metadata_post_covid = metadata_standardized.filter([
    pl.col("end_date").is_between(pl.datetime(2021, 6, 1, time_zone="UTC"), pl.datetime(3020, 3, 1, time_zone="UTC"))
])

metadata_post_covid.head()

id,start_date,end_date,cnae,postal_code,p1_kw,tarriff
str,date,date,str,str,f64,str
"""bcafdefe9c97b77d04b9deb42c97fe…",2021-06-01,2021-06-01,"""9820""","""48006""",5.0,"""2.0TD"""
"""5da6ad5cb04d01df2c70a7f1861da2…",2021-06-01,2021-06-01,"""9820""","""1003""",4.6,"""2.0TD"""
"""fbc6c7ccbc9de33c57c383b327030b…",2021-06-01,2021-06-01,"""9820""","""48280""",6.9,"""2.0TD"""
"""b4ce16d6fb68b1a439f18bea6d9e9d…",2021-06-01,2021-06-01,"""9820""","""20014""",3.3,"""2.0TD"""
"""a47e6497a647000d7b6fb49ccf3e40…",2021-06-01,2021-06-16,"""9820""","""48480""",3.3,"""2.0TD"""


In [9]:
metadata_post_covid_households = metadata_post_covid.filter(
    (pl.col('cnae') > '9699'),
    (pl.col('cnae') < '9900')
)
# metadata_post_covid_households.write_csv('data/metadata_post_covid_households.csv')
metadata_post_covid_households

id,start_date,end_date,cnae,postal_code,p1_kw,tarriff
str,date,date,str,str,f64,str
"""bcafdefe9c97b77d04b9deb42c97fe…",2021-06-01,2021-06-01,"""9820""","""48006""",5.0,"""2.0TD"""
"""5da6ad5cb04d01df2c70a7f1861da2…",2021-06-01,2021-06-01,"""9820""","""1003""",4.6,"""2.0TD"""
"""fbc6c7ccbc9de33c57c383b327030b…",2021-06-01,2021-06-01,"""9820""","""48280""",6.9,"""2.0TD"""
"""b4ce16d6fb68b1a439f18bea6d9e9d…",2021-06-01,2021-06-01,"""9820""","""20014""",3.3,"""2.0TD"""
"""a47e6497a647000d7b6fb49ccf3e40…",2021-06-01,2021-06-16,"""9820""","""48480""",3.3,"""2.0TD"""
…,…,…,…,…,…,…
"""644be161153608f1694861746ea2de…",2021-06-01,2022-04-24,"""9820""","""41120""",4.6,"""2.0TD"""
"""abeb05b9b0461b9be4313f6e6378c2…",2021-09-21,2022-04-25,"""9820""","""21200""",3.3,"""2.0TD"""
"""f275f372d4bb766f56a6529c77dcb1…",2021-06-01,2022-05-02,"""9820""","""42150""",4.6,"""2.0TD"""


# Analyze user data

In [10]:
# metadata_post_covid_households = pl.read_csv('data/metadata_post_covid_households.csv', try_parse_dates=True, schema_overrides= {'cnae':pl.String, 'postal_code':pl.String, 'start_date':pl.Date, 'end_date':pl.Date, 'p1_kw':pl.Float64, 'p2_kw':pl.Float64, 'p3_kw':pl.Float64, 'p4_kw':pl.Float64, 'p5_kw':pl.Float64, 'p6_kw':pl.Float64})

# metadata_post_covid_households = metadata_post_covid_households.with_columns(
#     pl.when(pl.col("postal_code").str.len_chars() == 4)
#       .then(pl.lit("0") + pl.col("postal_code"))
#       .otherwise(pl.col("postal_code"))
#       .alias("postal_code")
# )
#
# metadata_post_covid_households.head()

Start date analysis

In [11]:
cutoff = pl.datetime(2021, 6, 1)  # use pl.date(...) if your cols are Date

metadata_post_covid_households = (
    metadata_post_covid_households
    .with_columns(
        pl.max_horizontal(pl.col("start_date"), cutoff).alias("ref")
    )
    .with_columns(
        (pl.col("end_date") - pl.col("ref")).dt.total_days().alias("days_from_ref_to_end")
    )
)

metadata_post_covid_households.select(["id", "start_date", "end_date", "ref", "days_from_ref_to_end"])

id,start_date,end_date,ref,days_from_ref_to_end
str,date,date,datetime[μs],i64
"""bcafdefe9c97b77d04b9deb42c97fe…",2021-06-01,2021-06-01,2021-06-01 00:00:00,0
"""5da6ad5cb04d01df2c70a7f1861da2…",2021-06-01,2021-06-01,2021-06-01 00:00:00,0
"""fbc6c7ccbc9de33c57c383b327030b…",2021-06-01,2021-06-01,2021-06-01 00:00:00,0
"""b4ce16d6fb68b1a439f18bea6d9e9d…",2021-06-01,2021-06-01,2021-06-01 00:00:00,0
"""a47e6497a647000d7b6fb49ccf3e40…",2021-06-01,2021-06-16,2021-06-01 00:00:00,15
…,…,…,…,…
"""644be161153608f1694861746ea2de…",2021-06-01,2022-04-24,2021-06-01 00:00:00,327
"""abeb05b9b0461b9be4313f6e6378c2…",2021-09-21,2022-04-25,2021-09-21 00:00:00,216
"""f275f372d4bb766f56a6529c77dcb1…",2021-06-01,2022-05-02,2021-06-01 00:00:00,335


In [12]:
out = (
    metadata_post_covid_households
    .select(
        pl.col("days_from_ref_to_end")
          .value_counts()
          .sort()   # ascending
    )
    .unnest("days_from_ref_to_end")       # makes columns: days_from_... + count
    .with_columns(
        pl.col("count").reverse().cum_sum().reverse().alias("count_at_least_this_many_days ")
    )
)
out


# 4625 non-unique samples have end_date at least 365 days after 2021-06-01

days_from_ref_to_end,count,count_at_least_this_many_days
i64,u32,u32
-1,9,8764
0,46,8755
1,24,8709
2,22,8685
3,23,8663
…,…,…
995,5,16
996,1,11
997,3,10


Filter metadata by profiles with 365 days measurement after 2021-06-01

In [13]:
metadata_post_covid_households_year = metadata_post_covid_households.filter(pl.col('days_from_ref_to_end') >= 365)
metadata_post_covid_households_year

id,start_date,end_date,cnae,postal_code,p1_kw,tarriff,ref,days_from_ref_to_end
str,date,date,str,str,f64,str,datetime[μs],i64
"""dcd66a9ab6c5aa44c937dcf1afdcf4…",2021-06-01,2022-06-04,"""9820""","""36960""",4.4,"""2.0TD""",2021-06-01 00:00:00,368
"""56997980600a30317a357694a794cc…",2021-06-01,2022-06-03,"""9810""","""20250""",1.15,"""2.0TD""",2021-06-01 00:00:00,367
"""44f4ad91572bcd3c66fde8c5499af9…",2021-06-01,2022-06-03,"""9820""","""20002""",4.4,"""2.0TD""",2021-06-01 00:00:00,367
"""0cbeab5426e8f9b6270c8ebeb8d744…",2021-06-01,2022-06-05,"""9820""","""48991""",3.45,"""2.0TD""",2021-06-01 00:00:00,369
"""2afd32b695bc18ea99d36587c8390d…",2021-06-01,2022-06-03,"""9820""","""20560""",5.75,"""2.0TD""",2021-06-01 00:00:00,367
…,…,…,…,…,…,…,…,…
"""3b8c797107b8da562cf8e249e9f998…",2021-11-08,2023-03-16,"""9820""","""14011""",4.5,"""2.0TD""",2021-11-08 00:00:00,493
"""a885c9a6018a129e1abf0c6fa60752…",2021-11-12,2023-05-12,"""9820""","""42001""",4.6,"""2.0TD""",2021-11-12 00:00:00,546
"""6bfc974649312b08d73c45e82dde67…",2021-06-01,2022-10-13,"""9820""","""21207""",3.3,"""2.0TD""",2021-06-01 00:00:00,499


In [21]:
group_cols = [c for c in metadata_post_covid_households_year.columns
              if c not in ["start_date", "end_date", "ref", "days_from_ref_to_end"]]

metadata_post_covid_households_year = (
    metadata_post_covid_households_year
    .with_columns(
        pl.when(pl.col("start_date") < cutoff)
          .then(cutoff)
          .otherwise(pl.col("start_date"))
          .cast(pl.Date)
          .alias("start_date")
    )
    .with_columns(
        pl.max_horizontal(pl.col("start_date"), cutoff).alias("ref")
    )
    .with_columns(
        (pl.col("end_date") - pl.col("ref")).dt.total_days().alias("days_from_ref_to_end")
    )
    # choose row with max days_from_ref_to_end per group
    .sort(group_cols + ["days_from_ref_to_end"], descending=[False]*len(group_cols) + [True])
    .group_by(group_cols, maintain_order=True)
    .head(1)
)

metadata_post_covid_households_year = metadata_post_covid_households_year.select(['id', 'start_date', 'end_date', 'cnae','postal_code', 'p1_kw', 'tarriff', 'days_from_ref_to_end'])

metadata_post_covid_households_year.select(["id", "start_date", "end_date", "days_from_ref_to_end"])

id,start_date,end_date,days_from_ref_to_end
str,date,date,i64
"""002b95326c8255f7d54f4da5a9fb27…",2021-11-17,2022-11-18,366
"""002da62dc54b7a789473182feddfad…",2021-06-01,2022-08-19,444
"""00372feea8da6dd7ee0a404bd7f3bd…",2021-06-01,2023-05-15,713
"""0050b980111bad9422575db977d181…",2021-06-01,2022-10-25,511
"""0056ba30d63bfaaaabfaf337b312ab…",2022-03-05,2023-05-17,438
…,…,…,…
"""ffc4e2630581bce53d9075c7455dfa…",2022-01-27,2023-05-17,475
"""ffd19f52a6fd57060368ae718393de…",2021-06-01,2023-10-24,875
"""ffe091e941a9d641abb55b40df222c…",2021-06-01,2023-05-31,729


In [22]:
dupe_groups = (
    metadata_post_covid_households_year
    .group_by(['id', 'cnae', 'tarriff'])
    .len()
    .filter(pl.col("len") > 1)
    .sort("len", descending=True)
)

dupe_groups

id,cnae,tarriff,len
str,str,str,u32
"""ef1dd4566b1a397902b6b658a06c8c…","""9820""","""2.0TD""",2
"""2a1bc3f7b9446f1e0f977af4a9fd74…","""9820""","""2.0TD""",2
"""615bf372f0ef5cb9a072b4c1a59a76…","""9820""","""2.0TD""",2
"""b319bd499d03059093342a7cd94e38…","""9820""","""2.0TD""",2
"""15af9c4b5db406707d27efddcb9c53…","""9820""","""2.0TD""",2
…,…,…,…
"""846a47c7e0729bb55f158efb838d98…","""9820""","""2.0TD""",2
"""107d12bf07816a0504de4b7d470990…","""9820""","""2.0TD""",2
"""6ce1075e99fe084ce5cbac009b5ab5…","""9820""","""2.0TD""",2


In [23]:
rows_with_dupe_ids = metadata_post_covid_households_year.join(dupe_groups.select("id"), on="id", how="inner")
rows_with_dupe_ids.sort(["id", "start_date"])

id,start_date,end_date,cnae,postal_code,p1_kw,tarriff,days_from_ref_to_end
str,date,date,str,str,f64,str,i64
"""0774829cba433be96cff7b98485cec…",2021-06-01,2022-11-02,"""9820""","""48100""",4.5,"""2.0TD""",519
"""0774829cba433be96cff7b98485cec…",2021-06-01,2022-09-29,"""9820""","""48100""",6.9,"""2.0TD""",485
"""107d12bf07816a0504de4b7d470990…",2021-06-01,2022-12-08,"""9820""","""20003""",3.4,"""2.0TD""",555
"""107d12bf07816a0504de4b7d470990…",2021-06-01,2022-08-16,"""9820""","""20003""",4.4,"""2.0TD""",441
"""1578c48245af79cefa837a93928e89…",2021-06-12,2023-08-02,"""9820""","""48520""",2.0,"""2.0TD""",781
…,…,…,…,…,…,…,…
"""eb424e1a68f44801b1bfc7c0c0f6f1…",2021-08-21,2024-01-09,"""9820""","""48640""",10.0,"""2.0TD""",871
"""ef1dd4566b1a397902b6b658a06c8c…",2021-06-01,2022-07-01,"""9820""","""39180""",2.8,"""2.0TD""",395
"""ef1dd4566b1a397902b6b658a06c8c…",2021-06-01,2023-04-14,"""9820""","""39180""",3.4,"""2.0TD""",682


In [24]:
metadata_post_covid_households.filter(pl.col("id") == "846a47c7e0729bb55f158efb838d98d4f3907461dd60e92a728ff464978fcaca")

id,start_date,end_date,cnae,postal_code,p1_kw,tarriff,ref,days_from_ref_to_end
str,date,date,str,str,f64,str,datetime[μs],i64
"""846a47c7e0729bb55f158efb838d98…",2021-06-01,2022-12-21,"""9820""","""29603""",6.9,"""2.0TD""",2021-06-01 00:00:00,568
"""846a47c7e0729bb55f158efb838d98…",2021-06-01,2022-12-21,"""9820""","""29600""",6.9,"""2.0TD""",2021-06-01 00:00:00,568


In [25]:
metadata_post_covid_households_year.filter(pl.col("id") == "846a47c7e0729bb55f158efb838d98d4f3907461dd60e92a728ff464978fcaca")

id,start_date,end_date,cnae,postal_code,p1_kw,tarriff,days_from_ref_to_end
str,date,date,str,str,f64,str,i64
"""846a47c7e0729bb55f158efb838d98…",2021-06-01,2022-12-21,"""9820""","""29600""",6.9,"""2.0TD""",568
"""846a47c7e0729bb55f158efb838d98…",2021-06-01,2022-12-21,"""9820""","""29603""",6.9,"""2.0TD""",568


After making sure that the duplicates are due to multiple profiles of the same household, we can keep only one of them for our analysis. This results in 3359 unique households with at least 365 days of measurement after 2021-06-01.

In [26]:
subset_cols = ["id"]

metadata_post_covid_households_year_clean = (
    metadata_post_covid_households_year
    .sort(["id", "days_from_ref_to_end"], descending=True)
    .unique(subset=subset_cols, keep="last", maintain_order=True)
)
metadata_post_covid_households_year_clean

id,start_date,end_date,cnae,postal_code,p1_kw,tarriff,days_from_ref_to_end
str,date,date,str,str,f64,str,i64
"""ffeddf4919e748beb50af7c6b73def…",2021-06-01,2022-12-19,"""9820""","""42153""",6.0,"""2.0TD""",566
"""ffe48aa8a0036fe7add026a1f836bf…",2021-06-01,2023-05-15,"""9820""","""1191""",1.15,"""2.0TD""",713
"""ffe091e941a9d641abb55b40df222c…",2021-06-01,2023-05-31,"""9820""","""31810""",9.0,"""3.0TD""",729
"""ffd19f52a6fd57060368ae718393de…",2021-06-01,2023-10-24,"""9820""","""1015""",3.45,"""2.0TD""",875
"""ffc4e2630581bce53d9075c7455dfa…",2022-01-27,2023-05-17,"""9820""","""48650""",4.6,"""2.0TD""",475
…,…,…,…,…,…,…,…
"""0056ba30d63bfaaaabfaf337b312ab…",2022-03-05,2023-05-17,"""9820""","""20800""",2.3,"""2.0TD""",438
"""0050b980111bad9422575db977d181…",2021-06-01,2022-10-25,"""9821""","""31243""",3.3,"""2.0TD""",511
"""00372feea8da6dd7ee0a404bd7f3bd…",2021-06-01,2023-05-15,"""9820""","""12469""",3.45,"""2.0TD""",713


In [27]:
metadata_post_covid_households_year.write_csv('data/metadata_post_covid_households_year.csv')

In [28]:
households = metadata_post_covid_households_year['id'].unique().to_list()
print(f"Post-COVID households={len(households)}")

random_house = households[int(random() * len(households))]
print(f"Random household id: {random_house}")

Post-COVID households=3359
Random household id: 2672d9df3c28eff5c9746508a458393c3f09c0dbaef010861514172736d5d31e


In [29]:
random_house_df = pl.read_csv(EXTRACTED_DIR / f"{random_house}.csv", try_parse_dates=True, schema_overrides={'kWh':pl.Float64})
random_house_df

index,fl,kWh,imp
datetime[μs],i64,f64,i64
2019-07-25 00:00:00,1,0.121,0
2019-07-25 01:00:00,1,0.145,0
2019-07-25 02:00:00,1,0.074,0
2019-07-25 03:00:00,1,0.148,0
2019-07-25 04:00:00,1,0.088,0
…,…,…,…
2024-01-24 20:00:00,0,0.007,0
2024-01-24 21:00:00,0,0.003,0
2024-01-24 22:00:00,0,0.003,0


# Combine CSV files

In [ ]:
from csvmerger import CSVMerger
households_csvs = [f'{EXTRACTED_DIR}/{f}.csv' for f in households]

csvMerger = CSVMerger(households_csvs, None, 'data/household_kwh.csv')
csvMerger.combine_csv_files()

In [51]:
households_df = pl.read_csv('data/household_kwh.csv', try_parse_dates=True, schema_overrides={'kWh':pl.Float64}).with_columns(pl.col('index').alias('timestamp')).select('id', 'timestamp', 'kWh', 'imp')
# households_df

In [52]:
for col in households_df.columns:
    print(f"Column: {col}")
    print(households_df[col].value_counts().sort(by='count', descending=True).head(5))
    print("\n")

Column: id
shape: (5, 2)
┌─────────────────────────────────┬───────┐
│ id                              ┆ count │
│ ---                             ┆ ---   │
│ str                             ┆ u32   │
╞═════════════════════════════════╪═══════╡
│ 8c343670b22bd322a82aa326aa2fe5… ┆ 61969 │
│ 40d7a44464507e713b5a7953b2b899… ┆ 61273 │
│ 6ad819c80b9f9e6c1d76242b04c69d… ┆ 61105 │
│ a6ffcef47b13a29c82b5dd646408cd… ┆ 60625 │
│ 54c7533a24f659dda3777d9136b5a7… ┆ 60073 │
└─────────────────────────────────┴───────┘


Column: timestamp
shape: (5, 2)
┌─────────────────────┬───────┐
│ timestamp           ┆ count │
│ ---                 ┆ ---   │
│ datetime[μs]        ┆ u32   │
╞═════════════════════╪═══════╡
│ 2021-11-16 11:00:00 ┆ 3134  │
│ 2021-11-16 17:00:00 ┆ 3134  │
│ 2021-11-16 22:00:00 ┆ 3134  │
│ 2021-11-16 05:00:00 ┆ 3134  │
│ 2021-11-16 02:00:00 ┆ 3134  │
└─────────────────────┴───────┘


Column: kWh
shape: (5, 2)
┌───────┬─────────┐
│ kWh   ┆ count   │
│ ---   ┆ ---     │
│ f64   ┆ u32    

In [53]:
households_df

id,timestamp,kWh,imp
str,datetime[μs],f64,i64
"""8a0f77af6f419bb69998d048c27614…",2019-04-09 00:00:00,0.018,0
"""8a0f77af6f419bb69998d048c27614…",2019-04-09 01:00:00,0.019,0
"""8a0f77af6f419bb69998d048c27614…",2019-04-09 02:00:00,0.018,0
"""8a0f77af6f419bb69998d048c27614…",2019-04-09 03:00:00,0.019,0
"""8a0f77af6f419bb69998d048c27614…",2019-04-09 04:00:00,0.018,0
…,…,…,…
"""7d1ec51c8bfc18caa1f44952d6dcfc…",2022-06-27 19:00:00,0.069,0
"""7d1ec51c8bfc18caa1f44952d6dcfc…",2022-06-27 20:00:00,0.042,0
"""7d1ec51c8bfc18caa1f44952d6dcfc…",2022-06-27 21:00:00,0.038,0


In [56]:
post_covid_households_df = households_df.filter([pl.col('timestamp')
                                          .dt.replace_time_zone("UTC")
                                          .is_between(
                                            pl.datetime(2021, 6, 1, time_zone="UTC"),
                                            pl.datetime(3023, 1, 1, time_zone="UTC"))]).drop("imp")
post_covid_households_df

id,timestamp,kWh
str,datetime[μs],f64
"""8a0f77af6f419bb69998d048c27614…",2021-06-01 00:00:00,0.021
"""8a0f77af6f419bb69998d048c27614…",2021-06-01 01:00:00,0.02
"""8a0f77af6f419bb69998d048c27614…",2021-06-01 02:00:00,0.021
"""8a0f77af6f419bb69998d048c27614…",2021-06-01 03:00:00,0.02
"""8a0f77af6f419bb69998d048c27614…",2021-06-01 04:00:00,0.02
…,…,…
"""7d1ec51c8bfc18caa1f44952d6dcfc…",2022-06-27 19:00:00,0.069
"""7d1ec51c8bfc18caa1f44952d6dcfc…",2022-06-27 20:00:00,0.042
"""7d1ec51c8bfc18caa1f44952d6dcfc…",2022-06-27 21:00:00,0.038


In [57]:
post_covid_households_df.write_csv('data/post_covid_household_kwh.csv')